# Using EvoAug DeepSTARR model to test for distance dependent cooperativity between motifs
**Authorship:**
Adam Klie (last updated: *07/19/2023*)
***
**Description:**
This notebook uses a pretrained DeepSTARR model to show how to perform a distance dependent cooperativity analysis using EUGENe. We recapitulate panel b) of Figure 5 from the DeepSTARR paper: https://www.nature.com/articles/s41588-022-01048-5
***

In [ ]:
# Imports
import os
import torch
import numpy as np
from evoaug import evoaug
from evoaug_analysis import utils, model_zoo

# Load the model
The DeepSTARR model take ins length 249 nt sequences and the output is 2 scalar values representing the activity of developmental enhancers and housekeeping enhancers measured experimentally using STARR-seq.

In [ ]:
# Load the DeepSTARR model that was trained in the evo_aug paper
ckpt_path = "/cellar/users/aklie/projects/ML4GLand/models/deAlmeida22/evoaug/attr_analysis/DeepSTARR_baseline_1.ckpt"
deepstarr = model_zoo.DeepSTARR(2)
loss = torch.nn.MSELoss()
optimizer_dict = utils.configure_optimizer(deepstarr, lr=0.001, weight_decay=1e-6, decay_factor=0.1, patience=5, monitor='val_loss')
standard_cnn = evoaug.RobustModel(
    deepstarr, 
    criterion=loss, 
    optimizer=optimizer_dict, 
    augment_list=[]
)
# load model weights
standard_cnn = evoaug.load_model_from_checkpoint(standard_cnn, ckpt_path).eval()

# Load motifs: ETS, SREBP, GATA, AP-1 and Dref
See Supplementary table 10 of the DeepSTARR paper for the motif IDs (parially listed below). We will perform a distance-cooperativity analysis the four combinations of motifs listed in the paper:

1. ETS and SREBP
2. GATA and GATA
3. AP-1 and GATA
4. Dref and Dref

| TF motif | PWM model                                          | Collection name |
|----------|----------------------------------------------------|-----------------|
| AP-1     | jaspar__MA0476.1                                   | jaspar          |
| GATA     | flyfactorsurvey__srp_SANGER_5_FBgn0003507           | flyfactorsurvey |
| twist    | flyfactorsurvey__twi_da_SANGER_5_FBgn0000413        | flyfactorsurvey |
| Trl      | flyfactorsurvey__Trl_FlyReg_FBgn0013263             | flyfactorsurvey |
| SREBP    | flyfactorsurvey__HLH106_SANGER_10_FBgn0015234       | flyfactorsurvey |
| ETS      | flyfactorsurvey__Ets97D_SANGER_10_FBgn0004510       | flyfactorsurvey |
| Dref     | homer__AVYTATCGATAD_DREF                           | homer           |
| Ohler1   | homer__MYGGTCACACTG_Unknown1                       | homer           |
| Ohler6   | homer__AAAAATACCRMA_Unknown4                | homer           |

In [ ]:
# We can use motifdata for the jaspar and homer motif databases
import motifdata as md

In [ ]:
# From jaspar
ap1_motif = md.load_jaspar(motif_accs=["MA0476.1"])["MA0476.1"]
ap1_consensus = ap1_motif.consensus

# From flybase, had to do manual
gata_consensus = "TGATAAGG"
ets_consensus = "ACCGGAAGT"
srebp_consensus = "ATCACGCGAC"

# From homer
dref_motif = md.read_homer("/cellar/users/aklie/opt/homer/motifs/insects/dref.motif")["AVYTATCGATAD"]
dref_consensus = dref_motif.consensus

# Generate a name to consensus dictionary
name_to_consensus = {
    "AP-1": ap1_consensus,
    "GATA": gata_consensus,
    "ETS": ets_consensus,
    "SREBP": srebp_consensus,
    "DREF": dref_consensus
}

# Print out the consensus sequences
print(f"AP-1: {ap1_consensus}")
print(f"GATA: {gata_consensus}")
print(f"ETS: {ets_consensus}")
print(f"SREBP: {srebp_consensus}")
print(f"DREF: {dref_consensus}")

# Mode 1: ETS/SREBP -- synergize exclusively at close distances (<25 bp)

In [ ]:
import xarray as xr
import seqpro as sp
from eugene import interpret
from eugene import plot as pl

In [ ]:
# Analysis params
seq_number = 60
seq_len = 249
comparisons = [
    ("ETS", "SREBP"),
    ("GATA", "GATA"),
    ("AP-1", "GATA"),
    ("DREF", "DREF")
]

In [ ]:
random_backbones = sp.random_seqs((seq_number, seq_len), alphabet=sp.alphabets.DNA, seed=42)
sdata = xr.Dataset({"seq": (["_sequence", "_length"], random_backbones)})

In [ ]:
for comparison in comparisons:
    print(f"Running {comparison}")
    motif_a_name, motif_b_name = comparison

    # Run the GIA
    interpret.motif_distance_dependence_gia(
        model=standard_cnn,
        sdata=sdata,
        feature_A=name_to_consensus[motif_a_name],
        feature_B=name_to_consensus[motif_b_name],
        tile_step=1,
        style="deAlmeida22",
        seq_var="seq",
        results_var=f"{motif_a_name}_{motif_b_name}_cooperativity",
        distance_var=f"{motif_a_name}_{motif_b_name}_distance",
        device="cuda",
        batch_size=128
    )

    # Plot the results
    pl.distance_cooperativity_gia_plot(
        sdata,
        results_var=f"{motif_a_name}_{motif_b_name}_cooperativity",
        distance_var=f"{motif_a_name}_{motif_b_name}_distance",
        col_names=["Dev", "HK"],
        cols_to_plot=["Dev"],
        motif_a_name=motif_a_name,
        motif_b_name=motif_b_name
    )

# DONE!

---

# Scratch